## EY Data Challenge - Sentinel-1 Sample Notebook

In [1]:
import rasterio
import pandas as pd
import pystac_client
import rioxarray as rio
import planetary_computer
from datetime import datetime
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1/")

In [2]:
# Define the bounding box for the entire data region using (Latitude, Longitude)
# This is the region of New York City that contains our temperature dataset
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)

# Calculate the bounds for doing an archive data search
# bounds = (min_lon, min_lat, max_lon, max_lat)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0]) # (min_lon, min_lat, max_lon, max_lat)

# Define the time window - THIS IS THE PART THAT WE NEED TO ITERATE OVER LATER
time_window = "2021-06-01/2021-09-01"

In [3]:
search_sentinel_1 = stac.search(
    bbox=bounds, 
    datetime=time_window,
    collections=["sentinel-1-rtc"]
)
items_1 = list(search_sentinel_1.get_all_items())
print('This is the number of Sentinel-1 scenes that touch our region:', len(items_1))

signed_items_1 = [planetary_computer.sign(item).to_dict() for item in items_1]

# Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees
resolution = 10  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for crs=4326 

# Load the data using Open Data Cube
data_sentinel_1 = stac_load(items_1,bands=["vv", "vh"], patch_url=planetary_computer.sign, bbox=bounds, crs="EPSG:4326", resolution=scale)

c:\Users\GV631HJ\OneDrive - EY\Desktop\challenge\venv\Lib\site-packages\pystac_client\item_search.py:896: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


This is the number of Sentinel-1 scenes that touch our region: 7


In [4]:
median = data_sentinel_1.median(dim="time").compute()

In [5]:
filename = "../data/S1_median_2025-03-15_v1.tiff"

In [6]:
height = median.dims["latitude"]
width = median.dims["longitude"]

# Define the Coordinate Reference System (CRS) to be common Lat-Lon coordinates
# Define the tranformation using our bounding box so the Lat-Lon information is written to the GeoTIFF
gt = rasterio.transform.from_bounds(lower_left[1],lower_left[0],upper_right[1],upper_right[0],width,height)
median.rio.write_crs("epsg:4326", inplace=True)
median.rio.write_transform(transform=gt, inplace=True)

# Create the GeoTIFF output file using the defined parameters 
with rasterio.open(filename, 'w', driver='GTiff', width=width, height=height, crs='epsg:4326', transform=gt, count=2, compress='lzw', dtype='float64') as dst:
    dst.write(median.vv, 1)
    dst.write(median.vh, 2)
    dst.close()

C:\Users\GV631HJ\AppData\Local\Temp\ipykernel_12908\3859263188.py:1: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  height = median.dims["latitude"]
C:\Users\GV631HJ\AppData\Local\Temp\ipykernel_12908\3859263188.py:2: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  width = median.dims["longitude"]
